# UK - Datenanbindung Social Media Daten via Funnel - Daily SoMe Consolidate

* Um was handelt es sich hier  (Kurzbeschreibung Inhalt):
Die täglich von der FUnnel API abgezogenen Daten der Social Media Daten wurden in der DZ unter dem Schema 02_cleaned für die PZ-UK bereitgestellt.
Mit diesem notebook wird die Daily SoMe Consolidate Tabelle geupdatetd.

---
* QUELLEN:  
- Unity-Catalog:
- 03_transformed.linkedin_organic_daily_view
- 03_transformed.youtube_organic_post_daily_view
- 03_transformed.x_organic_daily_view
- 03_transformed.instagram_organic_post_daily
- 03_transformed.instagram_organic_stories_daily

* ZIEL:  
- Unity-Catalog:
- 03_transformed.consolidated_socials_daily


---
* Versionen (aktuelle immer oben):
- 07.10.2025 Minh Hieu Le: add columns which are relevant for calculating engagement
- 25.09.2025 Minh Hieu Le: correct data types of metric columns
- 29.07.2025 Justin Stange-Heiduk: Bug fixes
- 10.06.2025 Justin Stange-Heiduk: Init

#Imports

In [0]:
from pyspark.sql.functions import col, lit, to_date, year, udf, concat_ws, when, trim
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, DoubleType
import pyspark.sql.functions as F
from pyspark.sql.functions import regexp_replace, trim, col

In [0]:
%run ../../common/nb_init

# Target Schema

In [0]:
target_schema_name = '03_transformed'
target_path = 'funnel'

#Configure Tables

In [0]:
relevant_tables = [
    {
        'table_name': '03_transformed.facebook_organic_daily',
        'id_column': 'PostID',
        'created_date': 'CreatedDate',
        'date': 'Date',
        'channel': 'Facebook',
        'url': 'PostURL',
        'post_message': 'PostMessage',
        'post_type': 'PostType',
        'impressions': 'PostTotalImpressionsLifetime',
        'total_reactions': 'TotalReactionsDiff',
        'total_comments': 'TotalPostCommentsDiff',
        'total_shares': 'TotalPostSharesDiff',
        'total_clicks': 'LinkClicksLifetimeDiff',
        'engagement_rate': 'EngagementRating',
        'engagaments': 'Engagement',
        'weighted_engagement': 'WeightedEngagements',
        'Owner': 'Kein Owner'
    },
    {
        'table_name': '03_transformed.instagram_organic_post_daily',
        'id_column': 'PostID',
        'created_date': 'CreatedDate',
        'date': 'Date',
        'channel': 'Instagram',
        'url': 'PostURL',
        'post_message': 'PostMessage',
        'post_type': 'PostType',
        'impressions': 'TotalImpressions',
        'total_reactions': 'TotalInteractionsDiff',
        'total_comments': 'TotalCommentsDiff',
        'total_shares': 'TotalSharesDiff',
        'total_saves': 'TotalSavedDiff',
        'engagement_rate': 'EngagementRating',
        'engagaments': 'Engagement',
        'weighted_engagement': 'WeightedEngagements',
        'Owner': 'Kein Owner'
    },
    {
        'table_name': '03_transformed.instagram_organic_stories_daily',
        'id_column': 'StoryID',
        'created_date': 'CreatedDate',
        'date': 'Date',
        'channel': 'Instagram_Storie',
        'url': 'Fehlt noch',
        'post_message': 'Caption',
        'post_type': 'Storie',
        'impressions': 'TotalImpressions',
        'total_comments': 'TotalRepliesDiff',
        'total_shares': 'TotalSharesDiff',
        'total_reach': 'TotalReachDiff',
        'engagement_rate': 'EngagementRating',
        'engagaments': 'Engagement',
        'weighted_engagement': 'WeightedEngagements',
        'Owner': 'Kein Owner'
    },
    {
        'table_name': '03_transformed.linkedin_organic_daily_view',
        'id_column': 'PostID',
        'created_date': 'CreatedDate',
        'date': 'Date',
        'channel': 'LinkedIn',
        'url': 'PostURL',
        'post_message': 'PostContent',
        'post_type': 'ContentType',
        'impressions': 'Impressions',
        'total_reactions': 'Likes',
        'total_comments': 'Comments',
        'total_shares': 'Shares',
        'total_clicks': 'Clicks',
        'engagement_rate': 'EngagementRateInPercent',
        'engagaments': 'Engagements',
        'weighted_engagement': 'WeightedEngagements',
        'Owner': 'Kein Owner'
    },
    {
        'table_name': '03_transformed.x_organic_daily_view',
        'id_column': 'PostID',
        'created_date': 'CreatedDate',
        'date': 'Date',
        'channel': 'X',
        'url': 'PostURL',
        'post_message': 'PostMessage',
        'post_type': 'PostType',
        'impressions': 'Impressions',
        'total_reactions': 'Likes',
        'total_comments': 'Replies',
        'total_shares': 'Reposts',
        'total_clicks': 'LinkClicks',
        'engagement_rate': 'EngagementRateInPercent',
        'engagaments': 'Engagements',
        'weighted_engagement': 'WeightedEngagements',
        'Owner': 'Kein Owner'
    },
    { # TODO: include YouTube, if descriptions are available
       'table_name': '03_transformed.youtube_organic_post_daily_view',
       'id_column': 'VideoID',
       'created_date': 'CreatedDate',
       'date': 'Date',
       'channel': 'YouTube',
       'url': 'VideoURL',
       'post_type': 'Video',
       'post_message': 'VideoDescription',
       'impressions': 'Views',
       'total_reactions': 'Likes',
       'total_dislikes': 'Dislikes',
       'total_comments': 'Comments',
       'total_shares': 'Shares',
       'minutes_watched': 'EstimatedMinutesWatched',
       'total_views': 'Views',
       'engagement_rate': 'EngagementRateInPercent',
       'engagaments': 'Engagements',
       'weighted_engagement': 'WeightedEngagements',
       'Owner': 'Owner'
    },
    # TODO: include genios print media, if available
    # TODO: include genios online media, if available
]

# Combine relevant Tables

In [0]:
schema = T.StructType([
    T.StructField("ID", T.StringType(), True),
    T.StructField("CreatedDate", T.StringType(), True),
    T.StructField("Date", T.StringType(), True),
    T.StructField("Channel", T.StringType(), True),
    T.StructField("URL", T.StringType(), True),
    T.StructField("Owner", T.StringType(), True),
    T.StructField("PostMessage", T.StringType(), True),
    T.StructField("PostType", T.StringType(), True),
    T.StructField("Impressions", T.IntegerType(), True),
    T.StructField("TotalReactions", T.IntegerType(), True),
    T.StructField("TotalDislikes", T.IntegerType(), True),
    T.StructField("TotalComments", T.IntegerType(), True),
    T.StructField("TotalShares", T.IntegerType(), True),
    T.StructField("TotalClicks", T.IntegerType(), True),
    T.StructField("TotalSaves", T.IntegerType(), True),
    T.StructField("MinutesWatched", T.IntegerType(), True),
    T.StructField("TotalViews", T.IntegerType(), True),
    T.StructField("TotalReach", T.IntegerType(), True),
    T.StructField("EngagementRate", T.DoubleType(), True),
    T.StructField("WeightedEngagement", T.DoubleType(), True),
    T.StructField("Engagement", T.DoubleType(), True),
])
df_consolidated = spark.createDataFrame([], schema=schema)

for t in relevant_tables:
    df = spark.read.table(f"datif_pz_uk_{env}.{t['table_name']}")

    if t['channel'] == "Facebook":
        post_type_column = F.col(t['post_type'])
        channel_column = F.lit(t['channel'])
        URL_column = F.col(t['url'])
        Owner_column = F.lit('Kein Owner')
        total_reactions = F.col(t['total_reactions'])
        total_dislikes = F.lit(0)
        total_comments = F.col(t['total_comments'])
        total_shares = F.col(t['total_shares'])
        total_clicks = F.col(t['total_clicks'])
        total_saves = F.lit(0)
        minutes_watched = F.lit(0)
        total_views = F.lit(0)
        total_reach = F.lit(0)

    elif t['channel'] == "Instagram":
        post_type_column = F.col(t['post_type'])
        channel_column = F.lit(t['channel'])
        URL_column = F.col(t['url'])
        Owner_column = F.lit('Kein Owner')
        total_reactions = F.col(t['total_reactions'])
        total_dislikes = F.lit(0)
        total_comments = F.col(t['total_comments'])
        total_shares = F.col(t['total_shares'])
        total_clicks = F.lit(0)
        total_saves = F.col(t['total_saves'])
        minutes_watched = F.lit(0)
        total_views = F.lit(0)
        total_reach = F.lit(0)

    # Youtube Posttype Hartcodierung
    elif t['channel'] == "YouTube":
        post_type_column = F.lit("Video")
        channel_column = F.lit(t['channel'])
        URL_column = F.col(t['url'])
        Owner_column = F.col(t['Owner'])
        total_reactions = F.col(t['total_reactions'])
        total_dislikes = F.col(t['total_dislikes'])
        total_comments = F.col(t['total_comments'])
        total_shares = F.col(t['total_shares'])
        total_clicks = F.lit(0)
        total_saves = F.lit(0)
        minutes_watched = F.col(t['minutes_watched'])
        total_views = F.col(t['total_views'])
        total_reach = F.lit(0)

    # Instagram Storie Posttype Hartcodierung
    elif t['channel'] == "Instagram_Storie":
        post_type_column = F.lit("Storie")
        URL_column = F.lit("Existiert nicht")
        channel_column = F.lit("Instagram")
        Owner_column = F.lit("Kein Owner")
        total_reactions = F.lit(0)
        total_dislikes = F.lit(0)
        total_comments = F.col(t['total_comments'])
        total_shares = F.col(t['total_shares'])
        total_clicks = F.lit(0)
        total_saves = F.lit(0)
        minutes_watched = F.lit(0)
        total_views = F.lit(0)
        total_reach = F.col(t['total_reach'])

    elif t['channel'] == 'LinkedIn':
        post_type_column = F.col(t['post_type'])
        channel_column = F.lit(t['channel'])
        URL_column = F.col(t['url'])
        Owner_column = F.lit("Kein Owner")
        total_reactions = F.col(t['total_reactions'])
        total_dislikes = F.lit(0)
        total_comments = F.col(t['total_comments'])
        total_shares = F.col(t['total_shares'])
        total_clicks = F.col(t['total_clicks'])
        total_saves = F.lit(0)
        minutes_watched = F.lit(0)
        total_views = F.lit(0)
        total_reach = F.lit(0)

    elif t['channel'] == "X":
        post_type_column = F.col(t['post_type'])
        # X Daten nur wenn der Post Type nicht none ist
        df = df.filter(F.col(t['post_type']).isNotNull())
        channel_column = F.lit(t['channel'])
        URL_column = F.col(t['url'])
        Owner_column = F.lit("Kein Owner")
        total_reactions = F.col(t['total_reactions'])
        total_dislikes = F.lit(0)
        total_comments = F.col(t['total_comments'])
        total_shares = F.col(t['total_shares'])
        total_clicks = F.col(t['total_clicks'])
        total_saves = F.lit(0)
        minutes_watched = F.lit(0)
        total_views = F.lit(0)
        total_reach = F.lit(0)
    
    df = df.select(
        F.col(t['id_column']).alias("ID"),
        F.col(t['created_date']).alias("CreatedDate"),
        F.col(t['date']).alias("Date"),
        channel_column.alias("Channel"),
        # F.lit(t['channel']).alias("Channel"),
        # F.col(t['url']).alias("URL"),
        URL_column.alias("URL"),
        Owner_column.alias("Owner"),
        F.col(t['post_message']).alias("PostMessage"),
        post_type_column.alias("PostType"),
        F.col(t['impressions']).alias("Impressions"),
        total_reactions.alias("TotalReactions"),
        total_dislikes.alias("TotalDislikes"),
        total_comments.alias("TotalComments"),
        total_shares.alias("TotalShares"),
        total_clicks.alias("TotalClicks"),
        total_saves.alias("TotalSaves"),
        minutes_watched.alias("MinutesWatched"),
        total_views.alias("TotalViews"),
        total_reach.alias("TotalReach"),
        F.col(t['engagement_rate']).alias("EngagementRate"),
        F.col(t['weighted_engagement']).alias("WeightedEngagement"),
        F.col(t['engagaments']).alias("Engagement")
    )
    
    df_consolidated = df_consolidated.unionByName(df)

#### Bug das bei Facebook manchmal der PostType Null ist obwohl es einer Exisitiert

In [0]:
# 1. Quelle laden mit expliziter Umbenennung der PostType-Spalte
df_posttype_source = spark.read.table(f"datif_pz_uk_{env}.03_transformed.consolidated_socials") \
    .select(
        F.col("ID").alias("JoinID"),
        F.col("PostType").alias("PostType_source")
    ) \
    .dropDuplicates(["JoinID"])

# 2. Join & PostType auffüllen, falls im Ziel Null
df_consolidated = df_consolidated.join(
    df_posttype_source,
    df_consolidated["ID"] == df_posttype_source["JoinID"],
    how="left"
).withColumn(
    "PostType",
    F.when(F.col("PostType").isNull(), F.col("PostType_source"))
     .otherwise(F.col("PostType"))
).drop("PostType_source", "JoinID")

### Alle https: in der Postmessage löschen https://enbw.visualstudio.com/DaTIF/_workitems/edit/4281898/

In [0]:
df_consolidated = df_consolidated.withColumn(
    "PostMessage",
    regexp_replace("PostMessage", r"https:\/\/\S+", "")
)

### Einheitliches PostType Zuordnung

In [0]:
post_type_mapping = {
    ("YouTube", "Video"): "Video",
    ("Facebook", "video_direct_response"): "Video",
    ("Facebook", "video_inline"): "Video",
    ("Facebook", "Ohne Post Type"): "Other",
    ("Facebook", "album"): "Image",
    ("Facebook", "share"): "Other",
    ("Facebook", "photo"): "Image",
    ("Facebook", "profile_media"): "Other",
    ("Facebook", "cover_photo"): "Other",
    ("Facebook", "multi_share"): "Carousel / Document",
    ("Facebook", "multi_share_no_end_card"): "Other",
    ("Instagram", "VIDEO"): "Video",
    ("Instagram", "IMAGE"): "Image",
    ("Instagram", "CAROUSEL_ALBUM"): "Carousel / Document",
    ("Instagram", "Storie"): "Story",
    ("X", "mixed"): "Image",
    ("X", "gif"): "Other",
    ("X", "image"): "Image",
    ("X", "video"): "Video",
    ("LinkedIn", "Article"): "Other",
    ("LinkedIn", "Repost"): "Other",
    ("LinkedIn", "Image"): "Image",
    ("LinkedIn", "Poll"): "Poll",
    ("LinkedIn", "Document"): "Carousel / Document",
    ("LinkedIn", "Video"): "Video"
}



def map_post_type(channel, post_type):
    key = (channel, post_type)
    return post_type_mapping.get(key, "Keine Zuordnung möglich")

map_post_type_udf = udf(map_post_type, StringType())


df_consolidated = df_consolidated.withColumn(
    "PostTypeGeneralized",
    map_post_type_udf(F.col("Channel"), F.col("PostType"))
)

df_consolidated = df_consolidated.drop("PostType")
df_consolidated = df_consolidated.withColumnRenamed("PostTypeGeneralized", "PostType")

## Themen Mapping und null Werte ersetzen

In [0]:
%python
df_consolidated_total = spark.sql(f"""
SELECT
    ID,
    StrategischesThema1,
    StrategischesThema2,
    StrategischesThema3,
    Themenbereich1,
    Themenbereich2,
    Themenbereich3
    from datif_pz_uk_{env}.03_transformed.consolidated_socials""")


df_consolidated = df_consolidated.drop(
    "StrategischesThema1", "StrategischesThema2", "StrategischesThema3",
    "Themenbereich1", "Themenbereich2", "Themenbereich3"
).join(df_consolidated_total.alias("total"), on="ID", how="left")

def replace_empty_and_null(df, column_name, replacement):
    return df.withColumn(
        column_name,
        when(col(f"total.{column_name}").isNull() | (trim(col(f"total.{column_name}")) == ""), replacement)
        .otherwise(col(f"total.{column_name}"))
    )

# Liste der Spalten und ihre gewünschten Fallbacks
replacements = {
    "StrategischesThema1": "Kein strategisches Thema",
    "StrategischesThema2": "Kein strategisches Thema",
    "StrategischesThema3": "Kein strategisches Thema",
    "Themenbereich1": "Kein Themenbereich",
    "Themenbereich2": "Kein Themenbereich",
    "Themenbereich3": "Kein Themenbereich",
}

# Anwenden auf das DataFrame
for col_name, replacement_text in replacements.items():
    df_consolidated = replace_empty_and_null(df_consolidated, col_name, replacement_text)


Bug in Themenbereiche

"Geothermie/Pumpspeicher" → "Geothermie / Pumpspeicher"
" / Pumpspeicherkraftwerke" → " / Pumpspeicherkraftwerke"
"Geothermie /  Pumpspeicher" → "Geothermie / Pumpspeicher"

In [0]:


# Spalten, auf die die Regel angewendet werden soll
columns_to_fix = ["Themenbereich1", "Themenbereich2", "Themenbereich3"]

for col_name in columns_to_fix:
    df_consolidated = df_consolidated.withColumn(
        col_name,
        regexp_replace(col(col_name), r"\s*/\s*", " / ")  # ersetzt alle "/"-Varianten durch " / "
    ).withColumn(
        col_name,
        regexp_replace(col(col_name), r"\s{2,}", " ")  # entfernt doppelte Leerzeichen
    ).withColumn(
        col_name,
        trim(col(col_name))  # entfernt führende/trailing Leerzeichen
    )


#Write Table

In [0]:
fn_overwrite_table(df_source=df_consolidated, target_schema_name=target_schema_name, target_table_name="consolidated_socials_daily", target_path=target_path)